In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import os
import matplotlib.pyplot as plt


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [3]:
use_pretrained_model = False
pretrained_model_name = ""
context_frame_size = 13

In [4]:
train_data = np.load('workdir/train_tr_MFCC_0_D_A_norm_data_context_{}.npy'.format(str(context_frame_size)))
train_labels = np.load('workdir/train_tr_MFCC_0_D_A_norm_labels_context_{}.npy'.format(str(context_frame_size)))
val_data = np.load('workdir/train_va_MFCC_0_D_A_norm_data_context_{}.npy'.format(str(context_frame_size)))
val_labels = np.load('workdir/train_va_MFCC_0_D_A_norm_labels_context_{}.npy'.format(str(context_frame_size)))
test_data = np.load('workdir/test_MFCC_0_D_A_norm_data_context_{}.npy'.format(str(context_frame_size)))
test_labels = np.load('workdir/test_MFCC_0_D_A_norm_labels_context_{}.npy'.format(str(context_frame_size)))

print("Shape of training, validation and test data and corresponding labels")
for data, labels in [(train_data, train_labels), (val_data, val_labels), (test_data, test_labels)]:
    print(data.shape, labels.shape)

Shape of training, validation and test data and corresponding labels
(13, 3, 13, 1359661) (1359661,)
(13, 3, 13, 147731) (147731,)
(13, 3, 13, 1527014) (1527014,)


#### Change order of columns

In [13]:
X_train = np.reshape(train_data, (context_frame_size*13*3, len(train_labels)))
X_train = np.transpose(X_train, (1, 0))
Y_train_num = train_labels
X_val = np.reshape(val_data, (context_frame_size*13*3, len(val_labels)))
X_val = np.transpose(X_val, (1, 0))
Y_val_num = val_labels
X_test = np.reshape(test_data, (context_frame_size*13*3, len(test_labels)))
X_test = np.transpose(X_test, (1, 0))
Y_test_num = test_labels

nb_classes = 64 #number of phonemes

Y_train = np_utils.to_categorical(Y_train_num, nb_classes)
Y_val = np_utils.to_categorical(Y_val_num, nb_classes)
Y_test = np_utils.to_categorical(Y_test_num, nb_classes)


print("Shape of re-ordered training, validation and test data and corresponding labels")
for data, labels in [(X_train, Y_train), (X_val, Y_val), (X_test, Y_test)]:
    print(data.shape, labels.shape)

Shape of re-ordered training, validation and test data and corresponding labels
(1359661, 507) (1359661, 64)
(147731, 507) (147731, 64)
(1527014, 507) (1527014, 64)


In [14]:
if not use_pretrained_model:
    model = Sequential()
    model.add(Dense(32, input_shape=(context_frame_size*13*3,)))
    model.add(Activation('relu'))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(1024))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.08, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    history = model.fit(X_train, Y_train, batch_size=64, epochs=25, verbose=2, validation_data=(X_val, Y_val))
    loss_and_metrics = model.evaluate(X_test, Y_test, verbose=0)
    print(loss_and_metrics)
    [print("Error rate:{}".format(1-loss_and_metrics[1]))]
    
else:
    from keras.models import load_model
    model = load_model('pretrained_model_name')

Train on 1359661 samples, validate on 147731 samples
Epoch 1/25
124s - loss: 15.8640 - acc: 0.0157 - val_loss: 15.8670 - val_acc: 0.0156
Epoch 2/25


KeyboardInterrupt: 

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

In [ ]:
#model.save('models/context_19_conv32_3_3_padding_pool2_2_2_conv64_3_3_padding_pool_2_2dropout_0.1_64dense_80.81.5acc.h5')